# Introduction to cenpy American Community Survey margin of error tools

In [2]:
import cenpy as cen
from cenpy.moe import analytic_utils
%matplotlib inline

## 1. Pull data from the Census API

Identify some variables for the demonstration

In [3]:
cols_raw = ['B01003_001', # Total Population
            'B03001_003', # Hispanic
            'B19313_001', # Aggregate Income
            'B15002_012', # Males with Some College, Less Than 1 Year
            'B15002_013', # Males with Some College, 1 Or More Years, No Degree
            'B15002_014'] # Males with Associate's Degree

We will need the published estimates and margins of error

In [4]:
# Estimates end in an "E" and MOEs end in an "M"
cols_E = [i+'E' for i in cols_raw]
cols_M = [i+'M' for i in cols_raw]

cols_E

['B01003_001E',
 'B03001_003E',
 'B19313_001E',
 'B15002_012E',
 'B15002_013E',
 'B15002_014E']

In [5]:
cols_M

['B01003_001M',
 'B03001_003M',
 'B19313_001M',
 'B15002_012M',
 'B15002_013M',
 'B15002_014M']

We will pull data from the 2014-2018 detailed tables from the ACS

In [6]:
api_database = 'ACSDT5Y2018'
api_conn = cen.remote.APIConnection(api_database)
cen.explorer.explain(api_database)

{'American Community Survey: 1-Year Estimates: Detailed Tables 5-Year': 'The American Community Survey (ACS) is an ongoing survey that provides data every year -- giving communities the current information they need to plan investments and services. The ACS covers a broad range of topics about social, economic, demographic, and housing characteristics of the U.S. population.  Summary files include the following geographies: nation, all states (including DC and Puerto Rico), all metropolitan areas, all congressional districts (114th congress), all counties, all places, and all tracts and block groups.  Summary files contain the most detailed cross-tabulations, many of which are published down to block groups. The data are population and housing counts. There are over 64,000 variables in this dataset.'}

The demostration will be for all census tracts in Coconino County, Arizona

In [7]:
g_unit = 'tract'
g_filter = {'state':'04', 'county':'005'} # Coconino County, Arizona

With all the pieces in place, we can now pull the data from the Census API

In [8]:
data_E = api_conn.query(cols_E, geo_unit=g_unit, geo_filter=g_filter)
data_M = api_conn.query(cols_M, geo_unit=g_unit, geo_filter=g_filter)

data_E[cols_E] = data_E[cols_E].astype('int') # convert downloaded data to ints
data_M[cols_M] = data_M[cols_M].astype('int') # convert downloaded data to ints

Here is what the estimates look like

In [9]:
data_E

,B01003_001E,B03001_003E,B19313_001E,B15002_012E,B15002_013E,B15002_014E,state,county,tract
0,4611,618,155911000,63,201,101,04,005,001101
1,8339,1989,200765900,197,390,198,04,005,001102
2,5193,598,182234100,205,297,90,04,005,001301
3,5633,845,162323000,170,419,97,04,005,001302
4,6735,824,322765600,49,262,337,04,005,002200
5,5891,641,186444600,305,838,169,04,005,002300
6,3795,58,56610200,28,139,56,04,005,942201
7,3467,55,42392000,38,106,15,04,005,942202
8,4726,24,75087400,43,221,123,04,005,944900
9,4438,108,47734200,64,147,76,04,005,945000


...and the MOEs

In [10]:
data_M

,B01003_001M,B03001_003M,B19313_001M,B15002_012M,B15002_013M,B15002_014M,state,county,tract
0,406,170,22714480,42,64,52,04,005,001101
1,615,345,25521201,129,170,118,04,005,001102
2,599,255,22397589,95,114,52,04,005,001301
3,681,200,18576594,86,140,66,04,005,001302
4,635,209,63593535,57,128,141,04,005,002200
5,740,234,23082343,132,221,87,04,005,002300
6,304,37,5737985,16,35,23,04,005,942201
7,321,38,4566029,17,29,13,04,005,942202
8,517,28,10494052,35,101,58,04,005,944900
9,302,60,5082556,33,44,35,04,005,945000


## 2. Computing margins of error on combined estimates 

The ACS publishes an MOE alongside each estimte. These MOEs give an understanding of how reliable that particulare estimate is. In this demonstration we show how to compute the MOE when multiple ACS estimates are combined. Three options are available in cenpy:

a. Analytic

b. Pseudo

c. Replicate

### a. Analytic

The analytic approach uses classic statistical theory to compute the MOEs. 

In this first example, we sum three columns to get the unpublished value for males with more than high school diploma, but less than a bachelors degree.

In [11]:
# Summing values
results_sum = analytic_utils.analytic_sum(data_E[['B15002_012E','B15002_013E','B15002_014E']], 
                                          data_M[['B15002_012M','B15002_013M','B15002_014M']])
results_sum

,est,moe
0,365,92.541882
1,785,243.854465
2,592,157.241852
3,686,177.064960
4,648,198.781287
5,1312,271.724125
6,223,44.833024
7,159,36.041643
8,387,121.614144
9,287,65.192024


Second, we compute per capita income, which is a _ratio_ of aggregate income to total population.

In [12]:
# Ratio
results_ratio = analytic_utils.analytic_ratio(data_E[['B19313_001E','B01003_001E']],
                                              data_M[['B19313_001M','B01003_001M']])
results_ratio

,est,moe
0,33812.838864,5755.941612
1,24075.536635,3538.230172
2,35092.258810,5914.981952
3,28816.438843,4797.096911
4,47923.622866,10467.668017
5,31649.057885,5581.939327
6,14917.048748,1927.167003
7,12227.285838,1736.695411
8,15888.150656,2819.843781
9,10755.790897,1359.142260


Finally, we compute the _proportion_ Hispanic residents are of the total population.

In [13]:
# Proportion
results_prop = analytic_utils.analytic_prop(data_E[['B03001_003E','B01003_001E']],
                                            data_M[['B03001_003M','B01003_001M']])
results_prop

,est,moe
0,0.134027,0.034929
1,0.238518,0.037446
2,0.115155,0.047274
3,0.150009,0.030524
4,0.122346,0.028808
5,0.108810,0.037296
6,0.015283,0.009672
7,0.015864,0.010862
8,0.005078,0.005899
9,0.024335,0.013418


### b. Pseudo 

The pseudo approach generates random draws from a distribution around each published estimate built using the published estimate and MOE. These random draws are used to computed the MOEs on the combined estimates.